In [1]:
import tensorflow as tf

Chargement du dataset MNIST (handwritten digits), cf [MNIST handwritten digit database](https://http://yann.lecun.com/exdb/mnist/)

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 
# Les 2 ensembles : d'apprentissage & de test

Construction d'un modèle séquentiel (perceptron multi-couches)


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), # lignes bout à bout (vecteurs de pixels simple de dim 28*28)
  tf.keras.layers.Dense(128, activation='relu'), # 128 neurones de la deuxième couche (dense = interconnexion à tous les neurones d'entrée) => calcule act(w^t*x) 
  tf.keras.layers.Dropout(0.2), # Taux d'activation aléatoire
  tf.keras.layers.Dense(10) # couche de sortie (le nombre predict)
])

Point sur les fonctions d'activation et les couches denses : [Medium](https://medium.com/datathings/dense-layers-explained-in-a-simple-way-62fe1db0ed75)
$$ 
Relu(x) = max(0,x) 
$$

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.44476116,  0.09657557, -0.38829654, -0.2726937 ,  0.15289758,
         0.11602432,  0.4824928 ,  0.6147514 , -0.7359195 ,  1.0290267 ]],
      dtype=float32)

Pour chaque exemple, le modèle retourne un vecteur de "log-odds", une pour chaque classe (chaque chiffre)


In [5]:
tf.nn.softmax(predictions).numpy()


array([[0.05246667, 0.0901536 , 0.05551442, 0.06231771, 0.09537695,
        0.09192413, 0.1326126 , 0.15136448, 0.03921345, 0.2290559 ]],
      dtype=float32)

=> Probas pour chaque classe

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Fonction de perte : Entropie croisée 

In [7]:
loss_fn(y_train[:1], predictions).numpy()



2.3867917

Entropie croisée : $-log(p)$ où $p$ la probabilité de la classe correcte. Sur le modèle non entrainé, les probas sont proches de l'uniforme ($p = \frac{1}{10}$), d'où la proba proche de $-log(1/10) \approx 2.3$

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy']) # Mesure de la précision

In [9]:
model.fit(x_train, y_train, epochs=5) 
# On entraîne le modèle sur les données d'entrainement (epochs = itérations) 


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2964 - accuracy: 0.9141
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1425 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1087 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0914 - accuracy: 0.9723
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0760 - accuracy: 0.9764


In [10]:
model.evaluate(x_test,  y_test, verbose=2) # On teste le modèle sur des données qu'il n'a jamais vu



313/313 - 0s - loss: 0.0733 - accuracy: 0.9768


[0.07330185174942017, 0.9768000245094299]

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])


On utilise la fonction [softmax](https://fr.wikipedia.org/wiki/Fonction_softmax) pour que le modèle retourne une probabilité

In [12]:
probability_model(x_test[:5])


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.3363889e-08, 5.3988245e-09, 6.8073530e-07, 6.2287356e-05,
        1.7633494e-11, 1.2202025e-07, 7.1811318e-14, 9.9991417e-01,
        1.5015533e-07, 2.2532047e-05],
       [3.5826175e-09, 1.6559721e-05, 9.9995804e-01, 2.1591317e-05,
        3.7178434e-13, 3.2382434e-07, 4.1306287e-09, 4.0601131e-15,
        3.4084676e-06, 5.2895436e-14],
       [3.7220278e-07, 9.9891746e-01, 6.6548208e-05, 3.0060276e-06,
        7.7427532e-05, 7.2771850e-06, 3.0370951e-05, 8.1941491e-04,
        7.7489800e-05, 7.9428474e-07],
       [9.9967003e-01, 4.5913556e-08, 4.8620554e-05, 2.8363741e-07,
        1.6970838e-06, 6.9490111e-06, 2.4787910e-04, 2.3527671e-05,
        6.3821757e-09, 9.3300139e-07],
       [8.5695530e-07, 1.4706056e-08, 9.8476494e-07, 1.4861707e-08,
        9.9858677e-01, 2.2356805e-07, 1.4700519e-06, 2.6844064e-05,
        1.3637518e-06, 1.3814239e-03]], dtype=float32)>